### RAG Introduction with Langchain/Chromadb and LLAMACPP (OPENAI-API)

Here we built a RAG Application using library provided functions. 
We we will use 
- [LLAMA.CPP](https://github.com/ggml-org/llama.cpp) as the engine
  to run embedding and large language models.
- [ChromaDB](https://www.trychroma.com/) as an open source vector
  database
- [LangChain](https://python.langchain.com) as prompt engineering, RAG
  framework

We implement a chatbot that shall correspond with a PDF file of the 
following article about the evolution of the omicron COVID-19 variant
in France: [Retrospective analysis of SARS-CoV-2 omicron invasion over delta in French regions in 2021–22: a status-based multi-variant model](https://bmcinfectdis.biomedcentral.com/articles/10.1186/s12879-022-07821-5)

As such we have downloaded the PDF version of the Article and converted it
to plain text with `pdftotext` which comes with the [Poppler library](https://poppler.freedesktop.org/). The resulting file is provided as `s12879-022-07821-5.txt

For this course we need to run models using the [LLAMA.CPP](https://github.com/ggml-org/llama.cpp) engine in the background.
We shall serve the `all-MiniLM-L6-v2` embedding model:
```
llama-server -m all-MiniLM-L6-v2-Q8_0.gguf --host localhost --port 8081 --embedding
```
and the `Llama-3.2-3B-Instruct` LLM:
```
llama-server -m Llama-3.2-3B-Instruct-Q8_0.gguf -c 10000 --host localhost --port 8080
``` 
with a maximum context size of 10000.

Here we start with a general initialization of our system. As multiple participents of this course run their code on the same node we want to generate random port numbers to provide our REST services. Further we obtain the systems an IP address to bind our REST servers (embedding, llm) to.

In [1]:
import random
import socket
ip = socket.gethostbyname(socket.gethostname())
embedding_port = str(random.randint(40000,50000))
llm_port = str(random.randint(40000,50000))

We launch the rest servers (llama-cpp) in background.

In [2]:
from subprocess import Popen, DEVNULL

public_root = '/leonardo/pub/userexternal/thaschka/'
llama_cpp_server = public_root + 'llama.cpp/bin/llama-server'
llm_model = public_root + 'llm/llama-3.1-8B-I-Q8.gguf'
embedding_model = public_root + 'embed/all-MiniLM-L6-v2-Q8_0.gguf' 

embedding_process = Popen([llama_cpp_server, 
                           '-m', embedding_model, 
                           '--host', ip, 
                           '--port', embedding_port, 
                           '--embedding'],
                          stdout=DEVNULL,
                          stderr=DEVNULL)

llm_process = Popen([llama_cpp_server,
                     '-m', llm_model,
                     '--host', ip,
                     '--port', llm_port,
                     '-c', '10000',
                     '-ngl', '32'],
                     stdout=DEVNULL,
                     stderr=DEVNULL)

Just to assure that we have the right encoding on the machine we set everything to UTF-8

In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
from textwrap import wrap
f = open(public_root + 's12879-022-07821-5.txt','r')
text = f.read()
wrapped_text = wrap(text,1000)
f.close()

We wrapped the text into chunks of containing a maximum of 1000 characters each. *wrap* stops at spaces, and hence the chunks are a bit smaller. We have obtained 53 such chunks.

Langchain provides us with the REST api to talk to our local llama.cpp models. 
As llama.cpp uses the same API as OPENAI we can use the OPENAI model. 
There are however some quirks with the embedding API, which should be compatible, 
but is not, as such we use langchain's LocalAIEmbeddings API for the embeddings,
which works together with our llama.cpp server. 

In [5]:
# install packages
!pip install langchain_localai

from langchain_openai import ChatOpenAI
from langchain_localai import LocalAIEmbeddings
llm = ChatOpenAI(openai_api_base='http://' + ip + ':' + llm_port + '/v1',
                openai_api_key="whatever")

Defaulting to user installation because normal site-packages is not writeable


Let us try thing out: 

In [6]:
print(llm.invoke("Hello how are you today?"))

content="I'm just a language model, I don't have emotions or feelings, but I'm functioning properly and ready to help you with any questions or tasks you may have! How about you? How's your day going so far?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 16, 'total_tokens': 63, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'b5209-d2b2031e', 'id': 'chatcmpl-KmzsjEuld9AYJA3kZhUm0UEseEeZfQcD', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--4473eeb8-ffd1-4ab9-8ba3-477c69b8107c-0' usage_metadata={'input_tokens': 16, 'output_tokens': 47, 'total_tokens': 63, 'input_token_details': {}, 'output_token_details': {}}


In [7]:
vectors = LocalAIEmbeddings(openai_api_base="http://" + ip + ":" +embedding_port + "/v1",
                            openai_api_key="forget",
                            model="all-MiniLM-L6-v2",
                            openai_api_version='v1',)

In [8]:
len(vectors.embed_documents(wrapped_text))

53

In the next step we have to setup a vector database. 
Here as opposed to our first sample we use chromadb, but a large choice
of databases would exist. Only execute this once or destroy your presist_directory
and restart the notebooks' kernel. 

In [9]:
# install packages
!pip install chromadb

from langchain.vectorstores import Chroma
vectordb = Chroma.from_texts(texts=wrapped_text, 
                             embedding=vectors, 
                             persist_directory="chroma_db")
retriever = vectordb.as_retriever(search_kwargs={"k": 10})

Defaulting to user installation because normal site-packages is not writeable


Just as in the last case we have to deal with a chat template.
Here we can use langchains' tools in order include history and contexts. 

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate([
    ("system", "<|start_header_id|>system<|end_header_id|> \n\n" + \
    "You are a helpful AI assistent in answering prompt, " + \
    "taking the following contexts into account " + \
    "as good as you can as you answer. \n\n" + \
    "Contexts:\n"
    "{context}<|eot_id|>"),
    MessagesPlaceholder("chat_history"),
    ("human","<|start_header_id|>user<|end_header_id|> {input}<|eot_id|> \n\n" + \
    "<|start_header_id|>assistent<|end_header_id>")])
                            

Here we build our retrieval chain, with the template above. For each question we retrieve corresponding documents from the chromadb vector database.

In [11]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(retriever,question_answer_chain) 


Finally we have also to handle the chat history and define a query function

In [12]:
chat_history = []

def chat_query(question):
    a = rag_chain.invoke({"input": question, "chat_history": chat_history})

    human_msg = "<|start_header_id|>user<|end_header_id|>" + question + "<|eot_id|>"
    ai_answer = "<|start_header_id|>assistent<|end_header_id|>" + a['answer'] + "<|eot_id|>"

    chat_history.extend([
        ("human", human_msg),
        ("ai", ai_answer)
        ])
    print(a['answer'])
    return a

Now we can run our chatbot and ask some questions related to our text.

In [13]:
a = chat_query("Tell me about the 20-day window of opportunity")

The 20-day window of opportunity is a time period used in the study to analyze the replacement of the delta variant by the omicron variant. It is defined as a 20-day period that starts 10 days before the inflection point, where the omicron variant surpasses 50% of reported samples.

The inflection point is the moment when the omicron variant starts to take over and become the dominant variant, replacing the delta variant. This point is determined by visual inspection of the time series data between December 1st, 2021, and January 31st, 2022, which shows that the invasion of the omicron variant occurs approximately in three weeks in all metropolitan French regions.

The 20-day window of opportunity is chosen because it is long enough to capture the dynamics of the replacement process, but short enough to focus on the critical period when the omicron variant is taking over. This window is applied to each region independently, and the study uses data from Santé Publique France to estimate

In [14]:
a = chat_query("At what dates did the 20-day window of opportunity happen in france?")

According to the text, the inflection point, which marks the beginning of the 20-day window of opportunity, occurred approximately three weeks after December 1st, 2021. This means that the inflection point would be around:

December 1st, 2021 + 3 weeks = December 22nd, 2021

The 20-day window of opportunity would then start 10 days before this point, which would be:

December 22nd, 2021 - 10 days = December 12th, 2021

However, the text also mentions that the 20-day window of opportunity was used to analyze the period between the beginning of December and the end of January, when the omicron variant replaced the delta variant.

Given that the inflection point occurred around December 22nd, 2021, the 20-day window of opportunity would have started around:

December 22nd, 2021 - 10 days = December 12th, 2021

And would have ended around:

December 22nd, 2021 + 20 days = January 11th, 2022

So, the 20-day window of opportunity occurred from approximately December 12th, 2021, to January 11

In [15]:
embedding_process.terminate()

In [16]:
llm_process.terminate()